### The dubo Python SDK

The full docs are at https://docs.dubo.gg

- DataFrame API:
  - [dubo.ask](#dubo.ask)
  - [dubo.chart](#dubo.chart)
- [Database API](#Database-API)

Free tier requests are rate-limited–if you need a higher rate limit please contact support@dubo.gg.

In [ ]:
import dubo
print("Dubo Version", dubo.__version__)

## dubo.ask

The dubo library includes convenience functions to run queries on top of Pandas DataFrames.

For example, we can load the US Census data below into a Pandas DataFrame and then run `dubo.ask` on top of it.

In [ ]:
import pandas as pd
import dubo


# Grab a subset of data grouped by ZIP code from the 2021 American Community Survey
DATA_URL = (
    "https://raw.githubusercontent.com/ajduberstein/"
    "geo_datasets/master/2021_5_yr_acs.csv"
)
census_df = pd.read_csv(DATA_URL)
census_df['zip_code'] = census_df['zip_code'].apply(lambda x: str(x).zfill(5))
census_df.head()

In [ ]:
dubo.ask("What's the most populous ZIP code in the United States?", census_df, verbose=True)

How does it work? Internally, the library converts your query into a web request to our backend, where it is translated to SQL based on a combination of OpenAI's GPT-4 and other models. Your dataframe is loaded into an in-memory [SQLite3 database](https://www.sqlite.org/index.html) and the SQL returned from the server is then executed in this SQLite instance.

In [ ]:
dubo.ask(
    "What are the ten ZIP codes with the largest Hispanic "
    "populations in the United States?",
    census_df,
    verbose=True,
)

In [ ]:
dubo.ask(
    "Where is the wealthiest place in the US that is not majority white?",
    census_df,
    verbose=True,
)

## dubo.chart

Generate charts, using either [pydeck.gl](https://pydeck.gl/) for maps or [Vega-Altair](https://altair-viz.github.io/gallery/index.html) for charts.

In [ ]:
dubo.chart(
    "A scatterplot of male vs female population, with substantial opacity on the dots. "
    "If a dot is more male than female, make it orange.", census_df, verbose=True)

We can also specify the chart type explicitly, rather than let dubo infer it, like we will do on this dataset of power plants.

In [ ]:
power_df = pd.read_csv("https://raw.githubusercontent.com/ajduberstein/geo_datasets/master/global_power_plant_database.csv")
power_df.tail()

In [ ]:
import dubo

dubo.chart(
    "A scatterplot of powerplants, zoomed out",
    power_df,
    verbose=True,
    chart_type='deck_gl',
    notebook_display=True
)

## Database API

By [contacting us](founders@dubo.gg), you can connect our product directly to a database and then query against it. The Database API is modular. You can run a full text-to-SQL pipeline and extract results, or you can simply generate the SQL and not execute it, or just grab the tables that would be relevant for a particular query. You also get higher quality SQL than would be available in our DataFrame library.

The example below operates on the 400+ tables of [MusicBrainz](https://musicbrainz.org/doc/MusicBrainz_Database), a crowd-sourced music catalog used in music players like Spotify.

In [ ]:
import dubo
from dubo.config import set_dubo_key
import pandas as pd

# Demo API key
dubo.config.set_dubo_key('...')
df = pd.DataFrame(dubo.query("How many songs belong to artists that began their careers in New York?").results_set)
df.head()

In [ ]:
# Just grab the raw SQL
dubo.generate_sql("How many songs belong to artists that began their careers in New York?", pretty=True)

In [ ]:
# Isolate to the tables that may be relevant for the query
print('\n'.join([
    tbl.schema_name + '.' + tbl.table_name for tbl in
    dubo.search_tables("How many songs belong to artists that began their careers in New York?")
]))